# Recurrence Relations

https://jakevdp.github.io/blog/2013/06/15/numba-vs-cython-take-2/

In [1]:
import numpy as np
from numba import jit
import cython
%load_ext Cython

np.random.seed(1234)
import pandas as pd
pd.options.display.max_rows=12
s = Series(np.random.randn(int(1e5)))
com = 20.0
pd.__version__

'0.18.1'

In [2]:
def python(s):
    output = Series(index=range(len(s)))

    alpha = 1. / (1. + com)
    old_weight = 1.0
    new_weight = 1.0
    weighted_avg = s[0]
    output[0] = weighted_avg
    
    for i in range(1,len(s)):
        v = s[i]
        old_weight *= (1-alpha)
        weighted_avg = ((old_weight * weighted_avg) + 
                        (new_weight * v)) / (old_weight + new_weight)
        old_weight += new_weight
        output[i] = weighted_avg
        
    return output

In [3]:
%%cython
cimport cython
@cython.wraparound(False)
@cython.boundscheck(False)
def _cython(double[:] arr, double com, double[:] output):
    cdef:
        double alpha, old_weight, new_weight, weighted_avg, v
        int i
    
    alpha = 1. / (1. + com)
    old_weight = 1.0
    new_weight = 1.0
    weighted_avg = arr[0]
    output[0] = weighted_avg
    
    for i in range(1,arr.shape[0]):
        v = arr[i]
        old_weight *= (1-alpha)
        weighted_avg = ((old_weight * weighted_avg) + 
                        (new_weight * v)) / (old_weight + new_weight)
        old_weight += new_weight
        output[i] = weighted_avg
        
    return output

In [4]:
def cython1(s):
    output = np.empty(len(s),dtype='float64')
    _cython(s.values, com, output)
    return Series(output)

In [5]:
def cython2(s):
    return s.ewm(com=com,adjust=True).mean()

In [6]:
@jit
def _numba(arr, output):
    alpha = 1. / (1. + com)
    old_weight = 1.0
    new_weight = 1.0
    weighted_avg = arr[0]
    output[0] = weighted_avg
    
    for i in range(1,arr.shape[0]):
        v = arr[i]
        old_weight *= (1-alpha)
        weighted_avg = ((old_weight * weighted_avg) + 
                        (new_weight * v)) / (old_weight + new_weight)
        old_weight += new_weight
        output[i] = weighted_avg
    

def numba(s):
 
    output = np.empty(len(s),dtype='float64')
    _numba(s.values, output)
    return Series(output)

In [7]:
result1 = python(s)
result2 = cython1(s)
result3 = cython2(s)
result4 = numba(s)
result1.equals(result2) and result1.equals(result3) and result1.equals(result4)

True

In [8]:
%timeit python(s)

1 loop, best of 3: 2.03 s per loop


In [9]:
%timeit cython1(s)

1000 loops, best of 3: 982 µs per loop


In [10]:
%timeit cython2(s)

100 loops, best of 3: 9.65 ms per loop


In [11]:
%timeit numba(s)

1000 loops, best of 3: 991 µs per loop
